In [1]:
#Change this if necessary
data_dir = 'data'
pretrained_word_embeddings_path = 'glove.6B.100d.word2vec_format.txt'

In [2]:
import pickle
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
import gensim
import utils
import numpy as np
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt

# Compare pretrained vs. custom word embeddings

* Compare the word embedings for individual words (try positive words, negative words, nouns, verbs)
* What similarities do you see and what differencies do you see?

In [3]:
#Load pretrained and custom word embeddings
pretrained = gensim.models.KeyedVectors.load_word2vec_format(pretrained_word_embeddings_path)
custom = pickle.load(open('custom_word_embeddings.pkl', 'rb'))

In [ ]:
#superman, like, phone, glass, best, macarthur

In [4]:
pretrained.most_similar(['macarthur'])

[('truman', 0.5855777263641357),
 ('eisenhower', 0.5315422415733337),
 ('roosevelt', 0.5190081596374512),
 ('blamey', 0.5178776979446411),
 ('peck', 0.517310380935669),
 ('marshall', 0.513713002204895),
 ('ulysses', 0.5110025405883789),
 ('dewey', 0.5108863711357117),
 ('woodrow', 0.4982849955558777),
 ('forrest', 0.4922660291194916)]

In [5]:
custom.most_similar(['macarthur'])

[('commander', 0.7538551688194275),
 ('melbourne', 0.7535319328308105),
 ('cia', 0.7280160784721375),
 ('lex', 0.7209477424621582),
 ('fetisov', 0.7092979550361633),
 ('president', 0.7064594030380249),
 ('prophet', 0.7037036418914795),
 ('lt', 0.7010306119918823),
 ('zanuck', 0.6982249021530151),
 ('brashear', 0.697640597820282)]

In [143]:
pretrained.most_similar(['rate'])

[('rates', 0.9191826581954956),
 ('inflation', 0.782524824142456),
 ('increases', 0.7770980596542358),
 ('rise', 0.7459971904754639),
 ('higher', 0.7445486783981323),
 ('increase', 0.7373809814453125),
 ('low', 0.7282389998435974),
 ('growth', 0.7270269393920898),
 ('unemployment', 0.7264870405197144),
 ('interest', 0.7184916138648987)]

In [144]:
custom.most_similar(['rate'])

[('feature', 0.5912576913833618),
 ('foremost', 0.5568559765815735),
 ('grade', 0.5476285219192505),
 ('rated', 0.5365808010101318),
 ('suck', 0.5346225500106812),
 ('glance', 0.5031107068061829),
 ('recommend', 0.49670708179473877),
 ('give', 0.4915865361690521),
 ('rank', 0.48843464255332947),
 ('avoid', 0.47730809450149536)]

In [7]:
pretrained.most_similar(['phone'])

[('telephone', 0.9113394021987915),
 ('cellphone', 0.8122149705886841),
 ('phones', 0.8030568361282349),
 ('mobile', 0.730746328830719),
 ('mail', 0.7292180061340332),
 ('internet', 0.7269454002380371),
 ('messages', 0.7129441499710083),
 ('wireless', 0.7091723680496216),
 ('telephones', 0.7049247622489929),
 ('cellular', 0.6969102621078491)]

In [6]:
custom.most_similar(['phone'])

[('coffee', 0.6591180562973022),
 ('bus', 0.6589465141296387),
 ('table', 0.6388458013534546),
 ('plane', 0.6384596824645996),
 ('accidentally', 0.6371499300003052),
 ('cell', 0.6335462331771851),
 ('truck', 0.6327635049819946),
 ('bathroom', 0.6303887367248535),
 ('bike', 0.6227496862411499),
 ('gas', 0.6225556135177612)]

In [6]:
custom.closer_than('man', 'woman')

[]

In [7]:
pretrained.closer_than('man', 'woman')

[]

In [13]:
pretrained.doesnt_match('best great recommand like'.split())

'best'

In [12]:
custom.doesnt_match('best great recommand like'.split())

'like'

In [9]:
pretrained.rank('phone', 'bathroom' )

1885

In [8]:
custom.rank('phone', 'bathroom' )

8

# Compare features for reviews

In [10]:
#Delete word embeddings to reduce memory overhead
pretrained = None
custom = None

indices, reviews, labels = utils.get_training_data(data_dir)

reviews_features_sparse = pickle.load(open('reviews_features_sparse.pkl', 'rb'))
reviews_features_dense_pretrained = pickle.load(open('reviews_features_dense_pretrained.pkl', 'rb'))
reviews_features_dense_custom = pickle.load(open('reviews_features_dense_custom.pkl', 'rb'))

num_neighs = 5
metric = 'cosine'
nbrs_sparse = NearestNeighbors(n_neighbors=num_neighs, algorithm='auto', metric=metric).fit(reviews_features_sparse)
nbrs_pretrained = NearestNeighbors(n_neighbors=num_neighs, algorithm='auto', metric=metric).fit(reviews_features_dense_pretrained)
nbrs_custom = NearestNeighbors(n_neighbors=num_neighs, algorithm='auto', metric=metric).fit(reviews_features_dense_custom)

In [23]:
idx = 150 #Try changing this to see nearest neighbors for different reviews

#See nearest neighbors for sparse features
print(reviews[idx])
print("Sentiment: ", labels[idx])
distances, indices = nbrs_sparse.kneighbors(reviews_features_sparse[idx])
print("\nNearest neighbors")
for index, distance in zip(indices[0][1:], distances[0][1:]):
    print(reviews[index])
    print("Sentiment:", labels[index])
    print("Distance:", distance)
    print()

I wasn't sure what to expect but am I glad I went to see this. A smart, slightly twisted comedy that makes you think. I wasn't quite sure how a director can create "nothing", but leave it to Mr. Natali and the brilliant individuals at C.O.R.E. to create another low budget set that looks real (as real as nothing can be). Well worth your time and money, if you have the opportunity to see this, please go. You'll be glad you did.
Sentiment:  1

Nearest neighbors
I was glad to watch this movie free of charge as I am working in the hotel industry and this movie came lately to our movie library. Nothing against low budget movies, but this movie has horrible acting and directing. How can a movie as this one ever be made. The director should be blacklisted, and for all the poor actors, it is for sure not a jumping board into a career. Please make sure that you'll not watch this movie, the acting is lame, the camera and directing awful. There are just a few more movies out there which deserve to

In [27]:
#See nearest neighbors for dense pretrained features
print(reviews[idx])
print("Sentiment: ", labels[idx])
distances, indices = nbrs_pretrained.kneighbors([reviews_features_dense_pretrained[idx]])
print("\nNearest neighbors")
for index, distance in zip(indices[0][1:], distances[0][1:]):
    print(reviews[index])
    print("Sentiment:", labels[index])
    print("Distance:", distance)
    print()

I wasn't sure what to expect but am I glad I went to see this. A smart, slightly twisted comedy that makes you think. I wasn't quite sure how a director can create "nothing", but leave it to Mr. Natali and the brilliant individuals at C.O.R.E. to create another low budget set that looks real (as real as nothing can be). Well worth your time and money, if you have the opportunity to see this, please go. You'll be glad you did.
Sentiment:  1

Nearest neighbors
I loved this show so much and I'm so incredibly sad its canceled i thought it came back too, but just two stupid weeks. Thats terrible. i hate how we never find out how everyone ends up. it sucks. Bring it back! ABC has stupid shows like Supernanny and whatnot but doesn't give time to good ones like Six Degrees. If they're complaining about ratings it was probably because they had a bad slot because this was truly a good show, something I could relate to and anticipated. JJ Abrams delivered, he's awesome, I wish ABC could just trus

In [28]:
#See nearest neighbors for dense custom features
print(reviews[idx])
print("Sentiment: ", labels[idx])
distances, indices = nbrs_custom.kneighbors([reviews_features_dense_custom[idx]])
print("\nNearest neighbors")
for index, distance in zip(indices[0][1:], distances[0][1:]):
    print(reviews[index])
    print("Sentiment:", labels[index])
    print("Distance:", distance)
    print()

I wasn't sure what to expect but am I glad I went to see this. A smart, slightly twisted comedy that makes you think. I wasn't quite sure how a director can create "nothing", but leave it to Mr. Natali and the brilliant individuals at C.O.R.E. to create another low budget set that looks real (as real as nothing can be). Well worth your time and money, if you have the opportunity to see this, please go. You'll be glad you did.
Sentiment:  1

Nearest neighbors
I liked it, i really did. Please don't think that i'm an idiot but i have to admit that i enjoyed this film. I expected it to be crap, it was crap, but sometimes its OK to relax and watch a crappy film that you don't have to concentrate too much on isn't it? I didn't expect any hidden meanings or morales, and there wasn't any, but that doesn't matter because i only watched it for entertainment, and it did entertain me throughout. Films like this are why the Ben Stillers (excusing 'there's something about Mary') and the Vince Vaughn